# Monitor climate change over Europe with land reanalysis data

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import fsspec
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymannkendall as mk
import shapely.geometry
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Time
year_start = 1997
year_stop = 2022

# External files
shapefile_url = "https://www.eea.europa.eu/data-and-maps/data/eea-reference-grids-2/gis-files/spain-shapefile/at_download/file"
observed_csv = (
    "observed-annual-average-mean-surface-air-temperature-of-spain-for-1901-2022.csv"
)

## Set request

In [ ]:
collection_id = "reanalysis-era5-land-monthly-means"
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "time": "00:00",
    "area": [44, -10, 36, 0],
}

## Download data and convert to Celsius

In [ ]:
ds = download.download_and_transform(collection_id, request, chunks={"year": 1})
da = ds["t2m"]
with xr.set_options(keep_attrs=True):
    da -= 273.15
da.attrs["units"] = "°C"

## Select and cut Spain map

In [ ]:
def clip_shapefile(data, shapefile_url):
    shapefile_crs = "EPSG:4326"
    with fsspec.open(f"simplecache::{shapefile_url}") as file:
        gdf = gpd.read_file(file, layer="es_100km").to_crs(shapefile_crs)

    data = data.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude")
    data = data.rio.write_crs(shapefile_crs)
    data_clip = data.rio.clip(
        gdf.geometry.apply(shapely.geometry.mapping), gdf.crs, drop=False
    )
    return data_clip


da = clip_shapefile(da, shapefile_url)

## Plot annual mean

In [ ]:
da_time_mean = diagnostics.time_weighted_mean(da)
plot.projected_map(
    da_time_mean.where(da_time_mean), projection=ccrs.PlateCarree(), cmap="YlOrRd"
)
_ = plt.title("")

## Plot annual spatial mean

In [ ]:
da_spatial_mean = diagnostics.spatial_weighted_mean(da)
da_annual_mean = diagnostics.annual_weighted_mean(da_spatial_mean)
trend, h, p, z, tau, s, var_s, slope, intercept = mk.original_test(da_annual_mean)

# Plot bars
ax = da_annual_mean.to_pandas().plot.bar()
ax.set_ylabel(f"{da_annual_mean.attrs['long_name']} [{da_annual_mean.attrs['units']}]")
ax.bar_label(ax.containers[0], rotation=90, fmt="%.2f", padding=2.5)
plt.show()

# Plot lines
da_annual_mean.plot(label="Data")
plt.plot(
    da_annual_mean["year"],
    np.arange(da_annual_mean.sizes["year"]) * slope + intercept,
    label="Trend Line",
)
plt.legend()
plt.grid()
plt.title("Annual mean")
plt.show()

# Print significance
is_significant = p < 0.05
print(f"The trend is{'' if is_significant else ' NOT'} significant.")
print(f"Trend: {slope:f} {da_annual_mean.attrs['units']}/year")

## Comparison with in-situ data

In [ ]:
# Read the CSV file into a DataFrame
observed = pd.read_csv(observed_csv)
mask = (observed["Category"] >= year_start) & (observed["Category"] <= year_stop)
observed = observed[mask]

# Trend and significance
trend, h, p, z, tau, s, var_s, slope, intercept = mk.original_test(
    observed["Annual Mean"]
)
is_significant = p < 0.05
print(f"The observed trend is{'' if is_significant else ' NOT'} significant.")
print(f"Trend: {slope:f} {da_annual_mean.attrs['units']}/year")

# bias
bias = np.mean(np.array(da_annual_mean - observed["Annual Mean"]))
print(f"Bias: {bias} {da_annual_mean.attrs['units']}/year")

# Plot the first line
plt.plot(
    observed["Category"],
    observed["Annual Mean"],
    label="In-situ temperature",
    color="blue",
    marker="o",
)

# Plot the second line
plt.plot(
    da_annual_mean["year"],
    da_annual_mean,
    label="ERA5 land temperature ",
    color="red",
    marker="x",
)

# Add labels and title
plt.xlabel("Year")
plt.ylabel("Temperature(°C)")
plt.title(" Annual temperature from ERA5-Land and in-situ temperature")

# Add legend
plt.legend()
plt.grid()